#  Inert doublet model

According to this  [bug report](http://stauby.de/sarah_userforum/viewtopic.php?f=5&t=50#p252), we need to change by hand the file [prtcls1.mdl](../micromegas/SimplifiedDMIDM/work/models/prtcls1.mdl) to be sure that the DM candidate appears as the first defined $Z_2$-particle. In this case:

` etR       |~etR    |~etR    |35 ...
etI       |~etI    |~etI    |36 ...
etp       |~etp    |~Etp    |37 ...`

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
from hep import *

Define functions to change from general basis to physical basis 

In [25]:
def phys_to_int( m_h,m_H,m_A,m_Hp,alpha,lambda6,lambda7,m12_2,tan_beta,v):
  '''
    See arXiv:
  '''
  v2=v*v
  
  beta=np.arctan(tan_beta)
  sb = np.sin(beta)
  sb2 = sb*sb
  cb = np.cos(beta)
  cb2 = cb*cb
  tb = np.tan(beta)
  ctb = 1./tb
  
  sa  = np.sin(alpha)
  sa2 = sa*sa
  ca  = np.cos(alpha)
  ca2 = ca*ca

  sba=np.sin(beta-alpha)
  cba = np.sqrt(1.-sba*sba)
  
  l1=(m_H*m_H*ca2+m_h*m_h*sa2-m12_2*tb)/v2/cb2-1.5*lambda6*tb+0.5*lambda7*tb*tb*tb
  l2=(m_H*m_H*sa2+m_h*m_h*ca2-m12_2*ctb)/v2/sb2+0.5*lambda6*ctb*ctb*ctb-1.5*lambda7*ctb
  l3=((m_H*m_H-m_h*m_h)*ca*sa+2.*m_Hp*m_Hp*sb*cb-m12_2)/v2/sb/cb-0.5*lambda6*ctb-0.5*lambda7*tb
  l4=((m_A*m_A-2.*m_Hp*m_Hp)*cb*sb+m12_2)/v2/sb/cb-0.5*lambda6*ctb-0.5*lambda7*tb
  l5=(m12_2-m_A*m_A*sb*cb)/v2/sb/cb-0.5*lambda6*ctb-0.5*lambda7*tb
  #m22_2 = -0.5/sb*(pow(m_h,2)*ca*sba+pow(m_H,2)*sa*cba)+m12_2*ctb
  sinba = sba    
  
  return l1,l2,l3,l4,l5

Define function to run official micromegas IDM


## Check one point

###  With SARAH implementation
Based in [Scotogenic model implementation](https://github.com/restrepo/Scotogenic) by Avelino Vicente. Model files in the [SARAH/Models/SimplifiedDM/IDM](../SARAH/Models/SimplifiedDM/IDM) folder of this repo. We use below the python [hep](./hep.py) module to automalically manage input/output SARAH-Toolbox files (in a similar way to SSP)

In [7]:
a=hep(MODEL='THDMIII')

`a-object` is an object with many attributes and methods. Use the tab to explore them. Some of them are
* a.Series: [pandas](http://pandas.pydata.org/) Series object with the "relevant" variables 
* a.LHA: Input LesHouces file as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runSPheno() -> a.LHA_out: return LHA output files as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runmicromegas() -> a.runSPheno() -> Updated the `a-object`  with micrOMEGAS "relevant" output

In [231]:
v=a.vev
ip=pd.Series({'m_h':125.6,'m_H':355,'m_A':355,'m_Hp':350,'sinba':0.1,'lambda6':5.6,'lambda7':0,'tanb':30,'m12_2':1E2,
             'm_tau':1.77669000,'m_b':4.18,'m_t':1.73500000E+02,'cotbU':1/10.}) 
beta=np.arctan(ip.tanb)
alpha=-np.arcsin(ip.sinba)+beta
l1,l2,l3,l4,l5=phys_to_int( ip.m_h,ip.m_H,ip.m_A,ip.m_Hp,alpha,ip.lambda6,ip.lambda7,ip.m12_2,ip.tanb,v)
devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)
a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1] ='%0.8E       # lambda1' %l1
a.LHA.blocks['MINPAR'][2] ='%0.8E       # lambda2' %l2
a.LHA.blocks['MINPAR'][3] ='%0.8E       # lambda3' %l3
a.LHA.blocks['MINPAR'][4] ='%0.8E       # lambda4' %l4
a.LHA.blocks['MINPAR'][5] ='%0.8E       # lambda5' %l5
a.LHA.blocks['MINPAR'][6] ='%0.8E       # lambda6' %ip.lambda6
a.LHA.blocks['MINPAR'][7] ='%0.8E       # lambda7' %ip.lambda7
a.LHA.blocks['MINPAR'][9] ='%0.8E       # m12_2' % ip.m12_2
a.LHA.blocks['MINPAR'][10] ='%0.8E       # tanb' %ip.tanb
a.LHA.blocks['EPSEIN'].entries[3,3]='%0.8E       # epsYE(3,3)' %(-ip.tanb*ip.m_tau*np.sqrt(2)/v)
a.LHA.blocks['EPSDIN'].entries[3,3]='%0.8E       # epsYD(3,3)' %(-ip.tanb*ip.m_b*np.sqrt(2)/v)
a.LHA.blocks['EPSUIN'].entries[3,3]='%0.8E       # epsYU(3,3)' %(ip.cotbU*ip.m_t*np.sqrt(2)/v)
a.runSPheno()
print a.LHA_out_with_comments.blocks['FLAVORKITQFV'].entries[200]

3.16820359E-04  # BR(B->X_s gamma)


In [232]:
ip.cotbU=-1.
a.LHA.blocks['EPSUIN'].entries[3,3]='%0.8E       # epsYU(3,3)' %(ip.cotbU*ip.m_t*np.sqrt(2)/v)
a.runSPheno()
print a.LHA_out_with_comments.blocks['FLAVORKITQFV'].entries[200]

3.16769923E-04  # BR(B->X_s gamma)


In [233]:
ip.cotbU=-2
a.LHA.blocks['EPSUIN'].entries[3,3]='%0.8E       # epsYU(3,3)' %(ip.cotbU*ip.m_t*np.sqrt(2)/v)
a.runSPheno()
print a.LHA_out_with_comments.blocks['FLAVORKITQFV'].entries[200]

3.16742184E-04  # BR(B->X_s gamma)


In [234]:
ip.cotbU=-4
a.LHA.blocks['EPSUIN'].entries[3,3]='%0.8E       # epsYU(3,3)' %(ip.cotbU*ip.m_t*np.sqrt(2)/v)
a.runSPheno()
print a.LHA_out_with_comments.blocks['FLAVORKITQFV'].entries[200]

3.16749797E-04  # BR(B->X_s gamma)


In [236]:
pd.Series(a.LHA.blocks['MINPAR'].entries)

1     -3.31089530E+01       # lambda1
2      2.04888705E+00       # lambda2
3      1.11028977E+01       # lambda3
4     -2.00628165E+00       # lambda4
5     -2.12257114E+00       # lambda5
6      5.60000000E+00       # lambda6
7      0.00000000E+00       # lambda7
9        1.00000000E+02       # m12_2
10        3.00000000E+01       # tanb
dtype: object

In [237]:
pd.Series(a.LHA_out_with_comments.blocks['MASS'].entries)

25    1.25600000E+02  # hh_1
35    3.55000000E+02  # hh_2
36    3.55000000E+02  # Ah_2
37    3.50000000E+02  # Hm_2
23      9.11887000E+01  # VZ
24     8.03497269E+01  # VWm
1     5.00000000E-03  # Fd_1
3     9.50000000E-02  # Fd_2
5     4.18000000E+00  # Fd_3
2     2.50000000E-03  # Fu_1
4     1.27000000E+00  # Fu_2
6     1.73500000E+02  # Fu_3
11    5.10998930E-04  # Fe_1
13    1.05658372E-01  # Fe_2
15    1.77669000E+00  # Fe_3
dtype: object

[Jupyter home](./draft.pdf)